In [1]:
import os
from Evaluation.Evaluator import EvaluatorHoldout
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
#import torch

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader

data_reader = Movielens1MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()



URM_train, URM_validation = split_train_in_two_percentage_global_sample( URM_all, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
#evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

"""
from Diffusion.DiffusionRecommender import DiffusionAutoencoderRecommender_OptimizerMask
folder_path= '/Users/lucaortolomo/Downloads/Movielens1M/models'
file_name='/DiffusionAutoencoderRecommender_best_model_last'

diffrec = DiffusionAutoencoderRecommender_OptimizerMask(URM_train, use_gpu= False)
diffrec.load_model(folder_path=folder_path, file_name=file_name)
"""


Movielens1M: Preloaded data not found, reading from original files...
Movielens1M: Loading original data
Movielens1M: Unable to find data zip file. Downloading...
Downloading: https://files.grouplens.org/datasets/movielens/ml-1m.zip
In folder: /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Data_manager_split_datasets/Movielens1M/ml-1m.zip
DataReader: Downloaded 100.00%, 5.65 MB, 2545 KB/s, 2 seconds passed
Movielens1M: Loading Interactions
Movielens1M: Loading Item Features genres


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 3114: invalid continuation byte

In [19]:
URM_train.shape[0]

6040

In [15]:
URM_train

<6040x3883 sparse matrix of type '<class 'numpy.float64'>'
	with 800167 stored elements in Compressed Sparse Row format>

In [17]:
#things needed from URM
n_users, n_items = URM_train.shape

In [18]:
n_users

6040

In [5]:
#fit parameters
epochs = 100
batch_size = 128 #forse piu di 8(?)
l2_reg = 1e-4 #??????
sgd_mode ='adam'
learning_rate = 1e-2
noise_timesteps = 1000
inference_timesteps = 500
start_beta = 0.0001
end_beta = 0.10
activation_function = torch.nn.ReLU



from Diffusion.architecture_utils import generate_autoencoder_architecture
import numpy as np
#specific fro autoencoder rec
encoding_size = 50
next_layer_size_multiplier = 2
max_parameters = np.inf
max_n_hidden_layers = 3
noise_timesteps = 100
"""
  Example, if encoding = 50, next layer multiplier = 3, max hidden layers = 2
    the architecture is:
    [n_items, 450, 150, 50, 150, 450, n_items]
    """
#->
encoder_architecture = generate_autoencoder_architecture(encoding_size, n_items, next_layer_size_multiplier, max_parameters, max_n_hidden_layers)[::-1]

In [6]:
def _get_optimizer(optimizer_label, model, learning_rate, l2_reg):

    if optimizer_label.lower() == "adagrad":
        return torch.optim.Adagrad(model.parameters(), lr = learning_rate, weight_decay = l2_reg*learning_rate)
    elif optimizer_label.lower() == "rmsprop":
        return torch.optim.RMSprop(model.parameters(), lr = learning_rate, weight_decay = l2_reg*learning_rate)
    elif optimizer_label.lower() == "adam":
        return torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = l2_reg*learning_rate)
    elif optimizer_label.lower() == "sgd":
        return torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay = l2_reg*learning_rate)
    else:
        raise ValueError("sgd_mode attribute value not recognized.")

In [7]:
from Diffusion.PositionalEncoding import SinusoidalPositionalEncoding
from Diffusion.NoiseSchedule import LinearNoiseSchedule
from Diffusion.DenoisingArchitectures import AutoencoderModel
from DiffusionLuca.DiffusionModel import DiffusionModel
#Init diffusion model

#Positional encoding (not an expert at all so just going to put it there)
positional_encoding = SinusoidalPositionalEncoding(embedding_size = n_items, device="cpu:0")
#Here i am going to use a LinearNoiseSchedule, since i understant i can rewrite it myself
noise_schedule = LinearNoiseSchedule(start_beta=0.0001, end_beta=0.10, device = "cpu:0")

denoiser_model = AutoencoderModel(encoder_architecture = encoder_architecture,
                                          activation_function = activation_function,
                                          use_batch_norm = False,
                                          use_dropout = False,
                                          dropout_p = 0.3,
                                          device="cpu:0")

model = DiffusionModel(denoiser_model, noise_schedule, positional_encoding, noise_timesteps).to(device = "cpu:0")
import scipy.sparse as sps
warm_user_ids = np.arange(0, n_users)[np.ediff1d(sps.csr_matrix(URM_train).indptr) > 0]
#fine init



In [8]:
print (model.denoiser_model)

AutoencoderModel(
  (_encoder_network): Sequential(
    (layer_0): Linear(in_features=3883, out_features=200, bias=True)
    (activation_0): ReLU()
    (layer_1): Linear(in_features=200, out_features=100, bias=True)
    (activation_1): ReLU()
    (layer_2): Linear(in_features=100, out_features=50, bias=True)
    (activation_2): ReLU()
  )
  (_decoder_network): Sequential(
    (layer_0): Linear(in_features=50, out_features=100, bias=True)
    (activation_0): ReLU()
    (layer_1): Linear(in_features=100, out_features=200, bias=True)
    (activation_1): ReLU()
    (layer_2): Linear(in_features=200, out_features=3883, bias=True)
    (activation_2): ReLU()
  )
)


In [9]:
#start fit
current_epoch_training_loss = 0
_optimizer = _get_optimizer(sgd_mode.lower(), model, learning_rate, l2_reg)
loss_list = []


In [10]:
import math
from tqdm.auto import tqdm
#start training
def training_loop(model, URM_train, warm_user_ids, n_epochs, noise_timesteps, batch_size, optimizer, show_progress_bar = True):
    for epoch in tqdm(range(n_epochs),desc=f"Training progress", colour = "#00ff00"):
        current_epoch_loss=0
        model.train()

        num_batches_per_epoch = math.ceil(len(warm_user_ids) / batch_size)
        iterator = tqdm(range(num_batches_per_epoch)) if show_progress_bar else range(num_batches_per_epoch)
#one_epoch
        for _ in iterator:
            user_batch = torch.LongTensor(np.random.choice(warm_user_ids, size=batch_size))
            #print(user_batch[0])
            # Transferring only the sparse structure to reduce the data transfer
            user_batch_tensor = URM_train[user_batch]
            #print(user_batch_tensor[0])

            user_batch_tensor = torch.sparse_csr_tensor(user_batch_tensor.indptr,
                                                        user_batch_tensor.indices,
                                                        user_batch_tensor.data,
                                                        size=user_batch_tensor.shape,
                                                        dtype=torch.float32,
                                                        device="cpu:0",
                                                        requires_grad=False).to_dense()

            # Clear previously computed gradients
            optimizer.zero_grad()

            # Sample timestamps
            t = torch.randint(0, noise_timesteps, (len(user_batch_tensor),), device="cpu:0", dtype=torch.long)

            # Compute prediction for each element in batch
            #the forward process returns the mse loss between prediction and target!!!!
            loss = model.forward(user_batch_tensor, t)

            # Compute gradients given current loss
            loss.backward()
            # torch.nn.utils.clip_grad_norm(self._model.parameters(), max_norm=1.0)

            # Apply gradient using the selected optimizer
            optimizer.step()

            current_epoch_loss += loss.item()


        print("Epoch {}, loss {:.2E}".format(epoch, current_epoch_loss))
        # self.loss_list.append(self.current_epoch_training_loss)



In [11]:
#training_loop(model=model, URM_train=URM_train, warm_user_ids=warm_user_ids,n_epochs=100, noise_timesteps=noise_timesteps, batch_size=batch_size, optimizer=_optimizer)

In [21]:
from Recommenders.NonPersonalizedRecommender import TopPop

toppo = TopPop(URM_train=URM_train)
toppo.fit()
result_df, _ = evaluator_validation.evaluateRecommender(toppo)
result_df.loc[10]

TopPopRecommender: URM Detected 203 ( 5.2%) items with no interactions.
EvaluatorHoldout: Processed 6032 (100.0%) in 1.77 sec. Users per second: 3410


PRECISION                     0.182609
PRECISION_RECALL_MIN_DEN      0.191058
RECALL                        0.066456
MAP                           0.112597
MAP_MIN_DEN                   0.116327
MRR                           0.377093
NDCG                          0.154938
F1                            0.097448
HIT_RATE                      0.665451
ARHR_ALL_HITS                 0.620683
NOVELTY                       0.022098
AVERAGE_POPULARITY            0.769477
DIVERSITY_MEAN_INTER_LIST     0.494777
DIVERSITY_HERFINDAHL           0.94947
COVERAGE_ITEM                 0.017512
COVERAGE_ITEM_HIT             0.017255
ITEMS_IN_GT                   0.893381
COVERAGE_USER                 0.998675
COVERAGE_USER_HIT              0.66457
USERS_IN_GT                   0.998675
DIVERSITY_GINI                0.005007
SHANNON_ENTROPY               4.585432
RATIO_DIVERSITY_HERFINDAHL    0.950244
RATIO_DIVERSITY_GINI          0.014306
RATIO_SHANNON_ENTROPY         0.424016
RATIO_AVERAGE_POPULARITY 

In [13]:
from Diffusion.DiffusionRecommender import DiffusionAutoencoderRecommender_OptimizerMask

modelP = DiffusionAutoencoderRecommender_OptimizerMask(URM_train, use_gpu= False)
modelP.fit(epochs= 240, learning_rate= 0.00022443058798716912, l2_reg= 1.9501901504153858e-05, batch_size= 128, start_beta= 0.00038565911164407826, end_beta= 0.12444974448281955, noise_timesteps= 429.80432866068713, inference_timesteps= 1, objective= 'pred_x0', sgd_mode= 'rmsprop', activation_function= 'GELU', encoding_size= 245, next_layer_size_multiplier= 5, max_n_hidden_layers= 5, max_parameters= 1000000000.0)
result_df, _ = evaluator_validation.evaluateRecommender(modelP)
result_df.loc[10]


DiffusionAutoencoderRecommender: URM Detected 203 ( 5.2%) items with no interactions.
DiffusionAutoencoderRecommender: Architecture: [3883, 245]
fit!!!
429


/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Diffusion/DiffusionRecommender.py:401: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:56.)
  user_batch_tensor = torch.sparse_csr_tensor(user_batch_tensor.indptr,


DiffusionAutoencoderRecommender: Epoch 1, loss 2.10E+01
DiffusionAutoencoderRecommender: Epoch 1 of 240. Elapsed time 2.35 sec
DiffusionAutoencoderRecommender: Epoch 2, loss 1.99E+01
DiffusionAutoencoderRecommender: Epoch 2 of 240. Elapsed time 4.57 sec
DiffusionAutoencoderRecommender: Epoch 3, loss 1.98E+01
DiffusionAutoencoderRecommender: Epoch 3 of 240. Elapsed time 6.76 sec
DiffusionAutoencoderRecommender: Epoch 4, loss 1.93E+01
DiffusionAutoencoderRecommender: Epoch 4 of 240. Elapsed time 8.90 sec
DiffusionAutoencoderRecommender: Epoch 5, loss 1.95E+01
DiffusionAutoencoderRecommender: Epoch 5 of 240. Elapsed time 11.01 sec
DiffusionAutoencoderRecommender: Epoch 6, loss 1.94E+01
DiffusionAutoencoderRecommender: Epoch 6 of 240. Elapsed time 13.17 sec
DiffusionAutoencoderRecommender: Epoch 7, loss 1.97E+01
DiffusionAutoencoderRecommender: Epoch 7 of 240. Elapsed time 15.32 sec
DiffusionAutoencoderRecommender: Epoch 8, loss 2.00E+01
DiffusionAutoencoderRecommender: Epoch 8 of 240. Ela

PRECISION                     0.351177
PRECISION_RECALL_MIN_DEN      0.373882
RECALL                        0.156558
MAP                           0.254568
MAP_MIN_DEN                   0.266347
MRR                           0.630789
NDCG                          0.324883
F1                            0.216568
HIT_RATE                      0.886936
ARHR_ALL_HITS                 1.202685
NOVELTY                       0.024015
AVERAGE_POPULARITY            0.507474
DIVERSITY_MEAN_INTER_LIST     0.929306
DIVERSITY_HERFINDAHL          0.992915
COVERAGE_ITEM                 0.212207
COVERAGE_ITEM_HIT             0.185939
ITEMS_IN_GT                   0.893381
COVERAGE_USER                 0.998675
COVERAGE_USER_HIT             0.885762
USERS_IN_GT                   0.998675
DIVERSITY_GINI                0.049236
SHANNON_ENTROPY               7.931511
RATIO_DIVERSITY_HERFINDAHL    0.993725
RATIO_DIVERSITY_GINI          0.140673
RATIO_SHANNON_ENTROPY         0.733429
RATIO_AVERAGE_POPULARITY 

In [14]:
"""
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask

compare = MultVAERecommender_OptimizerMask(URM_train=URM_train)
compare.fit(epochs=200)

result_df, _ = evaluator_validation.evaluateRecommender(compare)
result_df.loc[10]


SyntaxError: EOF while scanning triple-quoted string literal (2884270800.py, line 8)